In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_log_error
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression


# Load the datasets
train_data = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e4/test.csv')

# One-hot encoding for 'Sex' 
ohe = OneHotEncoder()
sex_encoded = ohe.fit_transform(train_data[['Sex']]).toarray()
sex_columns = ohe.get_feature_names_out(['Sex'])
sex_df = pd.DataFrame(sex_encoded, columns=sex_columns)

# Merge encoded columns and drop original 'Sex' column
train_data = pd.concat([train_data, sex_df], axis=1)
train_data.drop('Sex', axis=1, inplace=True)

#test set one-hot encoding
# One-hot encoding for 'Sex' 
ohe = OneHotEncoder()
sex_encoded = ohe.fit_transform(test_data[['Sex']]).toarray()
sex_columns = ohe.get_feature_names_out(['Sex'])
sex_df = pd.DataFrame(sex_encoded, columns=sex_columns)

# Merge encoded columns for test dataset and drop original 'Sex' column
test_data = pd.concat([test_data, sex_df], axis=1)
test_data.drop('Sex', axis=1, inplace=True)

#keep IDs from test data separate
test_ID = test_data['id']

#drop ID from test dataset
test_data.drop('id', axis=1, inplace=True)

# Define features and target
X = train_data.drop(['Rings', 'id'], axis=1)
y = train_data['Rings']

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1234)


In [ ]:
poly2 = PolynomialFeatures(degree=2)
model_poly2 = make_pipeline(poly2, LinearRegression())
model_poly2.fit(X_train, y_train)

# Predictions
y_pred_poly2 = model_poly2.predict(X_val)
y_pred_poly2 = np.maximum(y_pred_poly2, 0)

# RMSLE
rmsle_poly2 = np.sqrt(mean_squared_log_error(y_val, y_pred_poly2))
print(f'RMSLE (Polynomial Regression degree 2): {rmsle_poly2}')

poly3 = PolynomialFeatures(degree=3)
model_poly3 = make_pipeline(poly3, LinearRegression())
model_poly3.fit(X_train, y_train)

# Predictions
y_pred_poly3 = model_poly3.predict(X_val)
y_pred_poly3 = np.maximum(y_pred_poly3, 0)

# RMSLE
rmsle_poly3 = np.sqrt(mean_squared_log_error(y_val, y_pred_poly3))
print(f'RMSLE (Polynomial Regression degree 3): {rmsle_poly3}')

poly4 = PolynomialFeatures(degree=4)
model_poly4 = make_pipeline(poly4, LinearRegression())
model_poly4.fit(X_train, y_train)

# Predictions
y_pred_poly4 = model_poly4.predict(X_val)
y_pred_poly4 = np.maximum(y_pred_poly4, 0)

# RMSLE
rmsle_poly4 = np.sqrt(mean_squared_log_error(y_val, y_pred_poly4))
print(f'RMSLE (Polynomial Regression degree 4): {rmsle_poly4}')

poly5 = PolynomialFeatures(degree=5)
model_poly5 = make_pipeline(poly5, LinearRegression())
model_poly5.fit(X_train, y_train)

# Predictions
y_pred_poly5 = model_poly5.predict(X_val)
y_pred_poly5 = np.maximum(y_pred_poly5, 0)

# RMSLE
rmsle_poly5 = np.sqrt(mean_squared_log_error(y_val, y_pred_poly5))
print(f'RMSLE (Polynomial Regression degree 5): {rmsle_poly5}')

Degree 3 Polynomial Regression is a clear winner when running from degree 2 through degree 5.

In [ ]:
# Test set predictions
test_predictions_poly = model_poly3.predict(test_data)
test_predictions_poly = np.maximum(test_predictions_poly, 0) # Ensure predictions are >= 0

In [ ]:
# Prepare the submission file
#round the predictions to integers
submission = pd.DataFrame({
    'id': test_ID,  
    'Rings': test_predictions_poly.round().astype(int)
})

# Save to CSV
submission.to_csv('submission.csv', index=False)

print('Submission file created: submission.csv')



submission